In [7]:
import pandas as pd
import matplotlib.pyplot as plt
data_df = pd.read_csv("test.csv", index_col='no')
print("총 data size : ",data_df.shape[0] )

총 data size :  19537


In [8]:
data_df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
no,,,,,,,,,,,,,,
29306,18,?,245274,Some-college,10,Never-married,?,Own-child,White,Male,0,0,16,United-States
29307,29,Private,83003,HS-grad,9,Married-civ-spouse,Other-service,Wife,White,Female,0,0,40,United-States
29308,45,Private,35136,Bachelors,13,Married-civ-spouse,Tech-support,Husband,Black,Male,0,0,40,United-States
29309,42,Self-emp-not-inc,64631,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States
29310,41,Private,195821,Doctorate,16,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,1902,40,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48838,45,Self-emp-not-inc,116789,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,60,United-States
48839,48,Private,185079,HS-grad,9,Never-married,Exec-managerial,Not-in-family,White,Female,0,0,50,United-States
48840,63,Private,117473,Some-college,10,Married-civ-spouse,Prof-specialty,Husband,White,Male,4386,0,40,United-States


# education-num 처리

In [16]:
data_df[data_df['education-num'].isnull()].shape
data_df['education-num'].unique()

['C', 'H', 'U', 'MA', 'M', 'E']
Categories (6, object): ['E' < 'M' < 'H' < 'C' < 'U' < 'MA']

In [14]:
b = [0,3,6,9,11,13,40]
label = ['E','M','H','C','U','MA']
data_df['education-num'] =  pd.cut(data_df['education-num'], bins=b, labels=label)

TypeError: '<' not supported between instances of 'int' and 'str'

# workclass 처리

In [17]:
a = pd.concat([data_df.workclass.value_counts(), data_df.workclass[data_df.income==1].value_counts()], axis=1)
a['c'] = a.iloc[:,1] / a.iloc[:,0] * 100
a.sort_values(by='c', ascending=False)

a['c'] = a['c'].fillna(0)
level_dic = {
    0:[],
    1:[],
    2:[],
    3:[],
    4:[],
    5:[],
    6:[],
    7:[],
    8:[],
    9:[]
}
for ele in a.index:
    level = a.loc[ele]['c']
    if level < 5 : level_dic[0].append(ele)
    elif level < 10 : level_dic[1].append(ele)
    elif level < 15 : level_dic[2].append(ele)
    elif level < 20 : level_dic[3].append(ele)
    elif level < 25 : level_dic[4].append(ele)
    elif level < 30 : level_dic[5].append(ele)
    elif level < 35 : level_dic[6].append(ele)
    elif level < 40 : level_dic[7].append(ele)
    elif level < 45 : level_dic[8].append(ele)
    else : level_dic[9].append(ele)

print(level_dic)
def filter_country(X):
    for key, value in level_dic.items():
        if X in value:
            return key
data_df['workclass'] = data_df['workclass'].apply(filter_country)

AttributeError: 'DataFrame' object has no attribute 'income'